https://dacon.io/competitions/official/235864/codeshare/4229?page=1&dtype=recent


In [33]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_excel('news_label.xlsx', engine='openpyxl')

In [3]:
df = df.rename(columns={'dates':'date', 'titles':'title','contents':'content'})
df['content'] = df['title'] + df['content']

In [4]:
df = df.drop(columns=['근거', 'title'])

In [5]:
df['label'] = df['label'].astype(int)
df

,date,content,label
0,2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝]//[\n마스카라까지 지우...,1
1,2021.10.07,"[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑]//[""파괴적 커머스 시대...",1
2,2021.10.08,[株머니 매니저의 HOT종목]//[\n\n\n\n\n 씨에스윈드는 풍력발전기를 구성...,1
3,2021.10.11,"[김민수 더맘마 대표 ""동네마트와 '상생'이 경쟁력…30분 총알배송 목표""]//[\...",1
4,2021.10.13,"[""카카오 보고있나""…진격의 네이버페이, 中 최대 직구몰 손잡았다]//[[주간투자동...",1
...,...,...,...
274,2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']//[\n심사 승인 났는...",0
275,2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합...",1
276,2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']//[\n■국감 오르...",0
277,2022.09.29,[국내 이커머스 상장 1호 주인공 누가될까 [격변의 이커머스①]]//[\n이커머스 ...,0


### 데이터 정제하기

In [6]:
# 문자 외의 것들 제거
df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
# 혹시 모를 다중 공백 제거
df['content'] = df['content'].str.replace(' +',' ') 
df

C:\Users\hslio\AppData\Local\Temp\ipykernel_25048\2956849729.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^가-힣a-zA-Z0-9_]',' ')
C:\Users\hslio\AppData\Local\Temp\ipykernel_25048\2956849729.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(' +',' ')


,date,content,label
0,2021.10.05,톡톡 이상품 한빛코리아 다지워 네이처 클렌징 솝 마스카라까지 지우는 세정비누 피부...,1
1,2021.10.07,격동의 유통가 편의점 성장 정체 미래는 근거리 쇼핑 파괴적 커머스 시대 데이터 경...,1
2,2021.10.08,머니 매니저의 HOT종목 씨에스윈드는 풍력발전기를 구성하는 타워 제작 및 설치를 ...,1
3,2021.10.11,김민수 더맘마 대표 동네마트와 상생 이 경쟁력 30분 총알배송 목표 별도 물류센터...,1
4,2021.10.13,카카오 보고있나 진격의 네이버페이 최대 직구몰 손잡았다 주간투자동향 파운트 400...,1
...,...,...,...
274,2022.09.23,WCP 흥행 실패에 컬리 케이뱅크 상장 시기 안갯속 심사 승인 났는데 IPO 감감...,0
275,2022.09.26,먹거리 IT 네이처랩 오경주 대표 맛있고 건강한 주스 추천 서비스를 개발합니다 서...,1
276,2022.09.27,투자 늘렸는데 이제와서 퀵커머스마저 규제하나 초긴장 국감 오르는 유통업 새 먹거리...,0
277,2022.09.29,국내 이커머스 상장 1호 주인공 누가될까 격변의 이커머스 이커머스 IPO 대전 흑...,0


In [14]:
train = df

In [15]:
train, test = train_test_split(train)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
test.reset_index(inplace=True)

In [17]:
test

,index,date,content,label
0,12,2021.10.25,ET 일주일 앞으로 다가온 위드 코로나 유통주가 대세 비대면 시대에 매장 100곳...,1
1,165,2022.04.27,마켓인 프롭테크하면 무조건 뜬다 투자유치 부익부 빈익빈 IPO 앞둔 오아시스마켓 ...,0
2,179,2022.05.12,제값 못 받느니 말 바꾼다 올해만 6번째 IPO 철회 원스토어 등 들어서만 6곳 ...,0
3,10,2021.10.21,오아시스부터 컬리까지 이커머스 업계 대세된 렌털 난각번호 1번 계란 주세요 비싸도...,1
4,17,2021.10.30,새벽배송 업계 내년 목표 상장 레이스 본격화 새벽배송 업계가 내년에 일제히 기업공...,1
...,...,...,...,...
65,132,2022.03.16,오아시스 이마트도 출격 판 커지는 퀵커머스 대전 주문 즉시 배송 유통가 퀵커머스 ...,1
66,205,2022.06.21,벤처 투자 혹한기에 성장보다 수익 챙긴 스타트업 뜬다 놀줄 알고 쓸줄 아는 A세대...,0
67,18,2021.10.31,새벽배송 이커머스 상장도 새벽배송 주간사 선정 끝내고 속도전 새벽배송 1호 상장사...,1
68,131,2022.03.15,대기업 스타트업도 퀵커머스 준비 효율 높이는 배민 B마트 디지털데일리 이안나 기자...,0


from sklearn.model_selection import train_test_split
X = df['content']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

X_train

X_test

### 토큰화 및 TF-IDF(Term Frequency-Inverse Document Frequency) 벡터화

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

In [39]:
# tf-idf 벡터화
for max_features in range(100, 10000, 100):
    vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

    X = train['content']
    y = np.array(train.label)

    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

    X_train = vectorizer.fit_transform(X_train)
    X_eval = vectorizer.transform(X_eval)
    
    lgs = LogisticRegression(class_weight='balanced')
    naive = BernoulliNB()
    sgd = SGDClassifier(random_state=42, n_jobs=-1)
    rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
    svc = SVC(random_state=42)
    ada = AdaBoostClassifier(random_state=42)
    
    lgs.fit(X_train, y_train)
    naive.fit(X_train, y_train)
    sgd.fit(X_train, y_train)
    rfc.fit(X_train, y_train)
    svc.fit(X_train, y_train)
    ada.fit(X_train, y_train)
    
    predicted = lgs.predict(X_eval)
    predicted = naive.predict(X_eval)
    predicted = sgd.predict(X_eval)
    predicted = rfc.predict(X_eval)
    predicted = svc.predict(X_eval)
    predicted = ada.predict(X_eval)
    
    print(f"1. Logistic_egression      max_features: {max_features}, Accuracy: {lgs.score(X_eval, y_eval)}" )
    print(f"2. Naive_bayse             max_features: {max_features}, Accuracy: {naive.score(X_eval, y_eval)}" )
    print(f"3. SGD                     max_features: {max_features}, Accuracy: {sgd.score(X_eval, y_eval)}" )
    print(f"4. RandomForestClassifier  max_features: {max_features}, Accuracy: {rfc.score(X_eval, y_eval)}" )
    print(f"5. SVC                     max_features: {max_features}, Accuracy: {svc.score(X_eval, y_eval)}" )
    print(f"6. AdaBoost                max_features: {max_features}, Accuracy: {ada.score(X_eval, y_eval)}" )
    print('=======')

1. Logistic_egression      max_features: 100, Accuracy: 0.8095238095238095
2. Naive_bayse             max_features: 100, Accuracy: 0.5476190476190477
3. SGD                     max_features: 100, Accuracy: 0.5
4. RandomForestClassifier  max_features: 100, Accuracy: 0.6190476190476191
5. SVC                     max_features: 100, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 100, Accuracy: 0.6428571428571429
1. Logistic_egression      max_features: 200, Accuracy: 0.6190476190476191
2. Naive_bayse             max_features: 200, Accuracy: 0.5952380952380952
3. SGD                     max_features: 200, Accuracy: 0.5714285714285714
4. RandomForestClassifier  max_features: 200, Accuracy: 0.5952380952380952
5. SVC                     max_features: 200, Accuracy: 0.5238095238095238
6. AdaBoost                max_features: 200, Accuracy: 0.5714285714285714
1. Logistic_egression      max_features: 300, Accuracy: 0.6428571428571429
2. Naive_bayse             max_features:

1. Logistic_egression      max_features: 1900, Accuracy: 0.7380952380952381
2. Naive_bayse             max_features: 1900, Accuracy: 0.5952380952380952
3. SGD                     max_features: 1900, Accuracy: 0.6904761904761905
4. RandomForestClassifier  max_features: 1900, Accuracy: 0.6190476190476191
5. SVC                     max_features: 1900, Accuracy: 0.5952380952380952
6. AdaBoost                max_features: 1900, Accuracy: 0.6666666666666666
1. Logistic_egression      max_features: 2000, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 2000, Accuracy: 0.5952380952380952
3. SGD                     max_features: 2000, Accuracy: 0.5476190476190477
4. RandomForestClassifier  max_features: 2000, Accuracy: 0.7142857142857143
5. SVC                     max_features: 2000, Accuracy: 0.5952380952380952
6. AdaBoost                max_features: 2000, Accuracy: 0.5952380952380952
1. Logistic_egression      max_features: 2100, Accuracy: 0.7857142857142857
2. Naive_bay

1. Logistic_egression      max_features: 3700, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 3700, Accuracy: 0.6904761904761905
3. SGD                     max_features: 3700, Accuracy: 0.6428571428571429
4. RandomForestClassifier  max_features: 3700, Accuracy: 0.5952380952380952
5. SVC                     max_features: 3700, Accuracy: 0.5714285714285714
6. AdaBoost                max_features: 3700, Accuracy: 0.7380952380952381
1. Logistic_egression      max_features: 3800, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 3800, Accuracy: 0.6904761904761905
3. SGD                     max_features: 3800, Accuracy: 0.7142857142857143
4. RandomForestClassifier  max_features: 3800, Accuracy: 0.6190476190476191
5. SVC                     max_features: 3800, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 3800, Accuracy: 0.7142857142857143
1. Logistic_egression      max_features: 3900, Accuracy: 0.7619047619047619
2. Naive_bay

1. Logistic_egression      max_features: 5500, Accuracy: 0.7857142857142857
2. Naive_bayse             max_features: 5500, Accuracy: 0.6666666666666666
3. SGD                     max_features: 5500, Accuracy: 0.6428571428571429
4. RandomForestClassifier  max_features: 5500, Accuracy: 0.6428571428571429
5. SVC                     max_features: 5500, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 5500, Accuracy: 0.6190476190476191
1. Logistic_egression      max_features: 5600, Accuracy: 0.7857142857142857
2. Naive_bayse             max_features: 5600, Accuracy: 0.6666666666666666
3. SGD                     max_features: 5600, Accuracy: 0.6666666666666666
4. RandomForestClassifier  max_features: 5600, Accuracy: 0.6428571428571429
5. SVC                     max_features: 5600, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 5600, Accuracy: 0.6190476190476191
1. Logistic_egression      max_features: 5700, Accuracy: 0.7857142857142857
2. Naive_bay

1. Logistic_egression      max_features: 7300, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 7300, Accuracy: 0.6428571428571429
3. SGD                     max_features: 7300, Accuracy: 0.6904761904761905
4. RandomForestClassifier  max_features: 7300, Accuracy: 0.6190476190476191
5. SVC                     max_features: 7300, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 7300, Accuracy: 0.6190476190476191
1. Logistic_egression      max_features: 7400, Accuracy: 0.7619047619047619
2. Naive_bayse             max_features: 7400, Accuracy: 0.6428571428571429
3. SGD                     max_features: 7400, Accuracy: 0.7142857142857143
4. RandomForestClassifier  max_features: 7400, Accuracy: 0.6428571428571429
5. SVC                     max_features: 7400, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 7400, Accuracy: 0.6666666666666666
1. Logistic_egression      max_features: 7500, Accuracy: 0.7619047619047619
2. Naive_bay

1. Logistic_egression      max_features: 9100, Accuracy: 0.7857142857142857
2. Naive_bayse             max_features: 9100, Accuracy: 0.6904761904761905
3. SGD                     max_features: 9100, Accuracy: 0.6904761904761905
4. RandomForestClassifier  max_features: 9100, Accuracy: 0.6904761904761905
5. SVC                     max_features: 9100, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 9100, Accuracy: 0.5714285714285714
1. Logistic_egression      max_features: 9200, Accuracy: 0.7857142857142857
2. Naive_bayse             max_features: 9200, Accuracy: 0.6904761904761905
3. SGD                     max_features: 9200, Accuracy: 0.6666666666666666
4. RandomForestClassifier  max_features: 9200, Accuracy: 0.6904761904761905
5. SVC                     max_features: 9200, Accuracy: 0.5476190476190477
6. AdaBoost                max_features: 9200, Accuracy: 0.5714285714285714
1. Logistic_egression      max_features: 9300, Accuracy: 0.7857142857142857
2. Naive_bay

In [47]:
from xgboost import XGBClassifier

X = train['content']
y = np.array(train.label)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

X_train = vectorizer.fit_transform(X_train)
X_eval = vectorizer.transform(X_eval)

# XGBClassifier 클래스의 객체 생성
xgb = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=5)

# 모델 학습
xgb.fit(X_train, y_train, early_stopping_rounds=400, eval_set=evals, eval_metric="logloss",  verbose=True)

# 예측
predicted = xgb_wrapper.predict(X_eval)

# 평가 - 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_eval, pred)
print('인공 신경망 예측 정확도: {0:.4f}'.format(accuracy))


C:\Users\hslio\miniforge3\envs\lhs\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\hslio\miniforge3\envs\lhs\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


AssertionError: Must have at least 1 validation dataset for early stopping.

In [77]:
# 2600 - 3000 이 0.83으로 정확도가 가장 높게 나옴

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=2600)

X = train['content']
y = np.array(train.label)

X_train = vectorizer.fit_transform(X)
X_test = vectorizer.transform(test['content'])
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y)
predicted = lgs.predict(X_test)

X_train = vectorizer.fit_transform(train['content'])
X_test = vectorizer.transform(test['content'])
y = np.array(train.label)

# lgs = LogisticRegression(class_weight='balanced')
# lgs.fit(X_train, y)
# predicted = lgs.predict(X_test)
rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
rfc.fit(X_train, y)
predicted = rfc.predict(X_test)

In [78]:
predicted

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1])

In [79]:
a = predicted.tolist()
b = np.array(test.label).tolist()

In [80]:
c = [i for i,j in zip(a,b) if i == j]
print(len(c))

51


In [81]:
correct_ratio = len(c) / len(test) * 100
correct_ratio

72.85714285714285

In [28]:
news_feature = df.drop(columns = ['content'])

In [30]:
news_feature.to_csv('news_feature.csv')